# Question 1: 
Given some sample data, write a program to answer the following: click here to access the required data set

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

sneaker_df = pd.read_csv("dataset.csv")
sneaker_df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [2]:
sneaker_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB


From the above code cell it is evident that the dataset has no null entries and size of the dataset is 5000 rows and 7 columns.

In [3]:
sneaker_df_sorted = sneaker_df.sort_values(by='created_at').reset_index(drop = True)
sneaker_df_sorted.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1863,39,738,536,4,cash,2017-03-01 0:08:09
1,1742,39,910,268,2,cash,2017-03-01 0:10:19
2,3229,97,912,324,2,cash,2017-03-01 0:14:12
3,1268,80,798,290,2,credit_card,2017-03-01 0:19:31
4,2690,49,799,258,2,credit_card,2017-03-01 0:22:25


In [4]:
print(f"Unique shops: {sneaker_df['shop_id'].nunique()} and Unique users: {sneaker_df['user_id'].nunique()}")

Unique shops: 100 and Unique users: 301


In [5]:
mean = np.mean(sneaker_df['order_amount']) 
avg_itms = np.mean(sneaker_df['total_items'])
avg_sneaker_val = np.mean(sneaker_df['order_amount']/sneaker_df['total_items'])

print("The average order value according to the question is", mean)
print("The average orders according to the question is", avg_itms)
print("The average value  of 1 sneaker according to the question is", avg_sneaker_val)

The average order value according to the question is 3145.128
The average orders according to the question is 8.7872
The average value  of 1 sneaker according to the question is 387.7428


### So let find out those anomalies
### First lets focus on the number of items ordered

In [6]:
sneaker_df['total_items'].value_counts()

2       1832
1       1830
3        941
4        293
5         77
2000      17
6          9
8          1
Name: total_items, dtype: int64

##### Whoa there are 17 order for 2000 sneakers. Looks like we found one of the reasons for the price being high. Lets explore if is it being caused by only one user - so if it is then we can remove those entries to get closer to real valued (logical) AOV.

In [7]:
sneaker_df[sneaker_df['total_items'] == 2000]['shop_id'].value_counts()

42    17
Name: shop_id, dtype: int64

##### Since we cannot control how the consumer spends but we can make adjustments to better understand the problem. So, I am going to remove the entries with the shop ID 42 as its a extreme outlier (as compared to the items ordered by other shops) in order to strengthen our model for a better understanding of the problem. 

In [8]:
mod_sneaker_df = sneaker_df[sneaker_df['shop_id']!=42]

### Now lets move on to order value

In [9]:
temp = pd.pivot_table(mod_sneaker_df, values='order_amount', index='shop_id', aggfunc=np.sum).sort_values(by="order_amount", ascending=False).reset_index()
temp.head()

,shop_id,order_amount
0,78,2263800
1,89,23128
2,81,22656
3,6,22627
4,13,21760


In [10]:
shop_78 = mod_sneaker_df[mod_sneaker_df["shop_id"] == 78]
print(shop_78.shape)
shop_78.head()

(46, 7)


,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
160,161,78,990,25725,1,credit_card,2017-03-12 5:56:57
490,491,78,936,51450,2,debit,2017-03-26 17:08:19
493,494,78,983,51450,2,cash,2017-03-16 21:39:35
511,512,78,967,51450,2,cash,2017-03-09 7:23:14
617,618,78,760,51450,2,cash,2017-03-18 11:18:42


##### As it is evident from the above block of code that there is one shop with ID "78" which is either a wrong entry or is a luxury brand which made its way here or is just charging very high per unit sneaker value. So lets remove the entries with shop ID "78" to get much more real AOV. Having a $25725 sneaker, i would even be afraid to step out of the house let alone wear it.

In [11]:
mod_sneaker_df = mod_sneaker_df[mod_sneaker_df["shop_id"] != 78]

In [12]:
np.mean(mod_sneaker_df['order_amount']), np.mean(sneaker_df['order_amount'])

(300.1558229655313, 3145.128)

## Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 

#### Existing conditions:-
As mentioned in the question that the average of all the porder values is as above but unfortunately thats the wrong as long as commmon sense dictates i.e. having a sneaker valued at $387.74, well thats one hell of a luxury product. But there can also be some problems with the way we calculated the values.

#### Potential problems and what have been identified from above code:-
1. There can be a outlier in number of items ordered, meaning that there could be one or more entries where the items ordered are somewhere in the range of thousands while rest are in the comparatively very less.
    - Identification done:- There was a shop with ID 42 from where some orders were made with items in the ordered numbering to 2000 while rest of the orders had less than 10, making it a extreme oulier and thus removed it.
2. There can be a outlier in total value of the orders, meaning that there could be one or more entries where the total value of the order are extremely high like maybe in millions (just trying to create a analogy) while rest are in just few thousands.
    - Identification done:- There was a Shop with ID 78 which had a sneaker valued at $25725 (still cant get over it), removed its entires
3. Other problems could be like there has been multiple entries of the same order, in other words duplicate entries but we would need more data to make that conclusion thus ignoring this here.

#### After solving the above mentioned problems i was able to get the AOV of around $\$$300 as compared to $\$$3000+ which is a huge change

### Other Evaluation methods below
- Median (population score) - Consider a situation where the dataset is huge (same thing happening here) and contains extreme outliers - mean would become unstable and would go haywire but that is not the case with median as this will not be swayed by extreme outliers as much as mean.
- z-mean (Independent score) - "A z-score describes the position of a raw score in terms of its distance from the mean, when measured in standard deviation units. The z-score is positive if the value lies above the mean, and negative if it lies below the mean." [Source](https://www.simplypsychology.org/z-score.html). Z-Score is important because it is useful to standardized the values by converting them into z-scores.

## What metric would you report for this dataset? What is its value?
1. Median

In [13]:
sneaker_df['order_amount'].describe()

count      5000.000000
mean       3145.128000
std       41282.539349
min          90.000000
25%         163.000000
50%         284.000000
75%         390.000000
max      704000.000000
Name: order_amount, dtype: float64

Thus the median of the dataset or in other words the AOV is around $284 (which is pretty close to the value i found out after removing some outliers). But when we say and talk about median, we also say and talk about 1st and 3rd quartile which explain the distribution of the sample, here it is the order values, and are very helpful in catching outliers. 1st quartile is at 25% and the 3rd is at 75% and there values are depicted in the above table.

2. z-score

- How do you calculate the z-score?:-

The formula for calculating a z-score is is z = (x-μ)/σ, where x is the raw score, μ is the population mean, and σ is the population standard deviation.

In [14]:
sneaker_df['z_score'] = ((sneaker_df['order_amount'] - 3145.128) / 41282.539).round(2)
sneaker_df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,z_score
0,1,53,746,224,2,cash,2017-03-13 12:36:56,-0.07
1,2,92,925,90,1,cash,2017-03-03 17:38:52,-0.07
2,3,44,861,144,1,cash,2017-03-14 4:23:56,-0.07
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37,-0.07
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11,-0.07


In [15]:
sneaker_df[(sneaker_df['shop_id'] == 42)].head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,z_score
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00,16.98
40,41,42,793,352,1,credit_card,2017-03-24 14:15:41,-0.07
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00,16.98
308,309,42,770,352,1,credit_card,2017-03-11 18:14:39,-0.07
409,410,42,904,704,2,credit_card,2017-03-04 14:32:58,-0.06


In [16]:
sneaker_df[(sneaker_df['shop_id'] == 78)].head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,z_score
160,161,78,990,25725,1,credit_card,2017-03-12 5:56:57,0.55
490,491,78,936,51450,2,debit,2017-03-26 17:08:19,1.17
493,494,78,983,51450,2,cash,2017-03-16 21:39:35,1.17
511,512,78,967,51450,2,cash,2017-03-09 7:23:14,1.17
617,618,78,760,51450,2,cash,2017-03-18 11:18:42,1.17


In [17]:
mod_2_sneaker_df = sneaker_df[(sneaker_df['shop_id'] != 78) & (sneaker_df['shop_id'] != 42)]
mod_2_sneaker_df.shape, mod_2_sneaker_df['z_score'].mean()

((4903, 8), -0.06832347542321031)

Mean z-score value of the whole dataset is : -0.06832 while for some extreme outlier is greater 1 like for shop with ID 42 the z-score is 16.98 - indicating a extreme value and for shop with ID 78 the value is 1.17.

So if we keep a threshold of 0.5 and try and find the AOV then, we get:-

In [20]:
sneaker_df[sneaker_df['z_score']<0.5]['order_amount'].mean()

302.58051448247926

In [21]:
sneaker_df[sneaker_df['z_score']<1]['order_amount'].mean()

400.04338175948345

As it can be seen from the above 2 code cell blocks that the tighter threshold we use the closer we get to what modified value we found out after removing some anomalies from the dataset, as a z-score gets closer to 0 indicating the closeness of the value to mean.